## 연습문제 - 201810803 장서령

주의:

 - 다른 과제보다 많이 약 10% 배점합니다

 - 다른 학생과 협력하지 마세요.

 - 제한시간이 다른 과제에 비해 짧아요 (3.5일)



A, B 2인의 가위바위보 후 내기금액을 이긴 사람에게 지급하는 게임을 블록체인에 개발하세요.

게임 컨트랙은 Rsp, 구현할 함수는 다음과 같다. 그 외 필요로 하는 함수는 추가할 수 있다.



함수 | 설명

-----|-----

setA | 무작위로 가위, 바위, 보를 생성하고, 설정한다. ```1000 wei```씩 금액을 건다.

setB | 외부에서 직접 입력하여 설정한다. ```1000 wei```씩 금액을 건다.

play | 가위, 바위, 보의 승패를 정한다.

distributeBetAmount | 이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 ```1000 wei```를 가지게 된다.

getMatchResult() | "A wins", "B wins", "tie" 결과를 출력



노드에서 게임을 다음과 같이 진행하세요.

- 게임 전의 A잔고, B잔고, 컨트랙 잔고 출력

- A가 내기금액 걸고, 컴퓨터에서 무작위로 선택한 가위바위보

- B가 내기금액 걸고, 사용자가 입력한 가위바위보

- 승부내기

- 누가 이겼는지 출력

- 게임 끝나고 A잔고, B잔고, 컨트랙 잔고 출력

# 1단계 : 컨트랙 개발

In [52]:
%%writefile src/Rsp.sol
pragma solidity ^0.6.0;

contract Rsp { //32bit짜리 숫자를 생성하는 rand함수 만들기
    uint8 randA;
    uint8 randB;
    uint8 whatresult;
    string transresult="";
    address owner; //컨트랙 제작자
    mapping (address => uint) balanceOf; // 잔고를 저장하는 mapping. 잔고의 key는 address, address의 key는 uint
    constructor() public { // 생성자, 누구나 객체를 만들 수 있음
        owner = msg.sender;  
    }
    
    //무작위로 가위, 바위, 보를 생성하고, 설정한다. 1000 wei씩 금액을 건다.
    function setA(uint amount) payable public {
        require(msg.value == amount);
        balanceOf[msg.sender] += amount;
        randA = uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3);
        // 0 : 가위, 1 : 바위, 2 : 보
    } 

    //외부에서 직접 입력하여 설정한다. 1000 wei씩 금액을 건다.
    function setB(address addr, uint amount, uint8 _value) payable public {
        require(msg.value == amount);
        balanceOf[addr] += amount;
        randB = _value;
    }
    
    //가위, 바위, 보의 승패를 정한다.
    function play() public { //tie = 3, A win = 4, B win = 5
        if(randA==0) { //가위
            if(randB==0) { //가위
                whatresult= 3;
            }
            else if(randB==1){ //바위
                whatresult= 5;
            }
            else{ //B가 보
                whatresult= 4;
            }
        }
        else if(randA == 1) { //바위
            if(randB==0) { //가위
                whatresult= 4;
            }
            else if(randB==1) {
                whatresult= 3;
            }
            else {
                whatresult= 5;
            }
        }
        else if(randA==2){ //보
            if(randB==0){ //가위
                whatresult= 5;
            }
            else if(randB==1){
                whatresult= 4;
            }
            else {
                whatresult= 3;
            }
        }
        if(whatresult==3){
            transresult = "tie";
        }
        else if(whatresult==4){
            transresult = "A win";
        }
        else if(whatresult==5){
            transresult = "B win";
        }
    }
    
    //이긴 측에게 내기 금액을 분배한다. A가 이기면 B의 내기 금액 1000 wei를 가지게 된다.
    function distributeBetAmount(address payable receiver) public payable{
        if(whatresult==3){ //tie
            balanceOf[receiver];
            balanceOf[msg.sender];
        }
        else if(whatresult==4){ //A win
            balanceOf[receiver] -= 1000;
            balanceOf[msg.sender] += 1000;
        }
        else if(whatresult==5){ //B win
            balanceOf[msg.sender] -= 1000;
            balanceOf[receiver] += 1000;
        }
    }
    
    // 컨트랙 잔고 확인
    function getContract() public view returns(uint){
         return (address(this).balance);
    }
    
    //A의 잔고 확인
    function getA() public view returns(uint) {
        return (balanceOf[owner]); // A의 잔고 출력       
    }
    
    //B의 잔고 확인
    function getB(address addr) public view returns(uint) {
        return balanceOf[addr]; //B의 계좌 잔고를 읽어옴
    }
    
    //"A wins", "B wins", "tie" 결과를 출력
    function getMatchResult()public view returns(string memory) {
        return (transresult);
    }
    
    function getwhatresult() public view returns(uint) {
        return (whatresult);
    }
    
}

Overwriting src/Rsp.sol


# 2단계 : 컴파일

In [53]:
!solc --gas src/Rsp.sol


======= src/Rsp.sol:Rsp =======
Gas estimation:
construction:
   infinite + 606800 = infinite
external:
   distributeBetAmount(address):	44855
   getA():	2005
   getB(address):	1263
   getContract():	237
   getMatchResult():	infinite
   getwhatresult():	1150
   play():	infinite
   setA(uint256):	42294
   setB(address,uint256,uint8):	42137


In [54]:
!solc src/Rsp.sol --combined-json abi,bin > src/Rsp.json

# 3단계 : 컨트랙 배포

In [55]:
%%writefile src/RspDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));
var fs=require('fs');
var _str = fs.readFileSync("src/Rsp.json");
var _json = JSON.parse(_str)
//var _abiArray = JSON.parse(_json.contracts.sHello2.abi);
var _abiArray = JSON.parse(_json.contracts["src/Rsp.sol:Rsp"].abi);
//var _bin = _json.contracts.sHello2.bin;
var _bin="0x" + _json.contracts["src/Rsp.sol:Rsp"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);

    new web3.eth.Contract(_abiArray).deploy({data: _bin}).estimateGas(function(err, gas) {
        if(!err) console.log(">> gas: "+ gas); //gas를 산정하는 함수를 위에 넣어놓음 
    });

    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 898371}, function(err, transactionHash) {
                if(!err) console.log("hash: " + transactionHash); 
        })
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/RspDeploy.js


In [56]:
!node src/RspDeploy.js

Deploying the contract from 0xC920E93d60AFaC15d7e2ca2C3fcCF5307b3e227e
>> gas: 898371
hash: 0x9025bdf4d2220dbafdf7e96a5e7b1a5526826034d56e47f4cddfe4027aa4aa77
---> The contract deployed to: 0xD6544EE46863607196B617724209EAd57d3F9Ac2


# 4단계 : 사용

In [66]:
%%writefile src/RspUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8445"));       //ok
var fs=require('fs');
var _str = fs.readFileSync("src/Rsp.json");
var _json = JSON.parse(_str)
var _abiArray = JSON.parse(_json.contracts["src/Rsp.sol:Rsp"].abi);

async function doIt() {//arr 대신 _rspGame
    var _rspGame = new web3.eth.Contract(_abiArray, "0xD6544EE46863607196B617724209EAd57d3F9Ac2");
    const accounts = await web3.eth.getAccounts();
    console.log("<The Init Balance>"); //A,B,컨트랙 초기잔고 출력 
    const Ainit = await _rspGame.methods.getA().call();           
    const Binit = await _rspGame.methods.getB(accounts[1]).call(); 
    const Cinit = await _rspGame.methods.getContract().call();
    miner.start(1);admin.sleepBlocks(1);miner.stop(); 
    console.log("A : " + Ainit);
    console.log("Contract : " + Cinit);
    console.log("B : " + Binit);
    console.log("setA, setB---------");
   /* console.log("\n");
    await _rspGame.methods.setA("1000").send({from: accounts[0], gas:42294,value:1000}); //A 가위바위보
    await _rspGame.methods.setB(accounts[1],"1000","1").send({from: accounts[0], gas:42137,value:1000}); //B 가위바위보
    const A = await _rspGame.methods.getA().call();           
    const B = await _rspGame.methods.getB(accounts[1]).call(); 
    const C = await _rspGame.methods.getContract().call();
    console.log("<The Balance Before The Game>"); //A,B,컨트랙 게임 전 잔고 출력 
    console.log("A : " + A);
    //console.log("Contract : " + C);
    console.log("B : " + B);
    
    console.log("play----------------");
    await _rspGame.methods.play().send({from: accounts[0], gas:77777}); //승부내기
    console.log("\n");
    console.log("<The Match Result>");
    const Mresult = await _rspGame.methods.getMatchResult().call();
    console.log("result is... " + Mresult); //이긴사람 출력  
    const what = await _rspGame.methods.getwhatresult().call();   // 
    console.log("whatresult is "+what); //                  
    console.log("send the money-------"); 
    await _rspGame.methods.distributeBetAmount(accounts[1]).send({from: accounts[0], gas:44855});
    console.log("\n");
                       
    console.log("<The Last Balance>");
    const AafterBal = await _rspGame.methods.getA().call();
    const ContractafterBal = await _rspGame.methods.getB(accounts[1]).call(); 
    const BafterBal = await _rspGame.methods.getContract().call();
    console.log("A : " + AafterBal);
    console.log("Contract : " + ContractafterBal);
    console.log("B : " + BafterBal);*/
}

doIt()

Overwriting src/RspUse.js


In [67]:
!node src/RspUse.js

<The Init Balance>


(node:6000) UnhandledPromiseRejectionWarning: Error: Returned values aren't valid, did it run Out of Gas? You might also see this error if you are not using the correct ABI for the contract you are retrieving data from, requesting data from a block number that does not exist, or querying a node which is not fully synced.
    at ABICoder.decodeParameters (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-eth-abi\src\index.js:332:15)
    at Contract._decodeMethodReturn (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-eth-contract\src\index.js:572:22)
    at Method.outputFormatter (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-eth-contract\src\index.js:932:42)
    at Method.formatOutput (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-core-method\src\index.js:169:54)
    at sendTxCallback (C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-core-method\src\index.js:640:33)
    at C:\Users\15Z970-G.AA5BK\Code\jsr\node_modules\web3-core-requestmanager\src\index.js:177:9
    at XM